<a href="https://colab.research.google.com/github/almirars/MachineLearning/blob/main/uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- NIM  : 2241727003
- Nama : Almira Rahma Sabita
- Kelas :  TI 4J

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import numpy as np
import pandas as pd

In [25]:
data = pd.read_csv('/content/drive/MyDrive/ML/tweet_emotions.csv')

data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [26]:
# cek terdapat nilai yang kosong atau tidak
data.isnull().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

**Pra Pengolahan Data**
* Drop kolom tweet_id
* Remove username
* Remove tanda baca
* Remove links

**Drop Kolom**

In [27]:
data = data.drop(['tweet_id'],axis=1)
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


**Remove Username**

In [42]:
import re
import nltk
# nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from textblob import Word

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [43]:
STOPWORDS = stopwords.words("english")

In [44]:
def clean(text):
    text = text.lower()
    text = re.sub("[^\w\s]","",text) # Remove punctuations 
    text = " ".join(w for w in text.split() if w not in STOPWORDS)
    text = " ".join([Word(word).lemmatize() for word in text.split()])
    return text

In [45]:
data['content'] = data['content'].apply(lambda x : clean(x)) 

In [46]:
freq=  pd.Series(" ".join(data['content']).split()).value_counts()[-10000:] 

In [47]:
freq =  list(freq.index)

In [48]:
data['content'] = data['content'].apply(lambda x:  " ".join(x for x in x.split() if x not in freq))

In [49]:
data.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhhwaitin call
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


**Ekstrasi Fitur**

In [51]:
# Bag of Words (TF-IDF)

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

x = np.array(data['content'].values)[:20000]
y =  np.array(data['sentiment'].values)[:20000]

Le = LabelEncoder()
y =  Le.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=True,random_state=42,stratify=y)

tf = TfidfVectorizer(analyzer='word',max_features=1000,ngram_range=(1,3))
x_tf = tf.fit_transform(x_train)
x_val_tf = tf.transform(x_test)

x_tf   = x_tf.toarray()
x_val_tf =  x_val_tf.toarray()

**Pembuatan Model**

In [52]:
# menggunakan algoritma Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(x_tf, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(x_tf)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(x_val_tf)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.386125
Hasil akurasi data test: 0.33225


**Evaluasi**

In [53]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [54]:
print("MultinomialNB")
print('Accuracy  = ', round(accuracy_score(y_test, y_pred_test)*100,2),'%')
print('Recall    = ', round(recall_score(y_test, y_pred_test, average="weighted")*100,2),'%')
print('Precision = ', round(precision_score(y_test, y_pred_test, average="weighted")*100,2),'%')
print('F1-Score  = ', round(f1_score(y_test, y_pred_test, average="weighted")*100,2),'%')

print("")
print("MultinomialNB")
print(classification_report(y_test,y_pred_test))  

MultinomialNB
Accuracy  =  33.22 %
Recall    =  33.22 %
Precision =  32.2 %
F1-Score  =  25.12 %

MultinomialNB
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        52
           4       0.00      0.00      0.00        93
           5       0.00      0.00      0.00       180
           6       0.58      0.03      0.06       212
           7       0.33      0.01      0.01       158
           8       0.24      0.14      0.18       740
           9       0.00      0.00      0.00        87
          10       0.33      0.26      0.29       886
          11       1.00      0.00      0.01       212
          12       0.35      0.79      0.48      1243

    accuracy                           0.33      4000
   macro avg       0.22      0.10      0.08      4000
weighted avg       0.3

Model Sentiment Analysis menghasilkan akurasi sekitar 33.2 % untuk prediksi sentimen menggunakan Naive Bayes.